In [3]:
import pandas as pd
from pandas.io.json import json_normalize

In [4]:
# Assign JSON file to a DataFrame
df = pd.read_json('brexit8.txt')

In [5]:
df

,E07000026,E07000032,E07000223,E07000224,S12000033,S12000034,S12000035,S12000041
0,"{'leave': {'total': 31809, 'percent': 58.6}, '...","{'leave': {'total': 44501, 'percent': 60.3}, '...","{'leave': {'total': 20315, 'percent': 54.6}, '...","{'leave': {'total': 56936, 'percent': 62.5}, '...","{'leave': {'total': 40729, 'percent': 38.9}, '...","{'leave': {'total': 62516, 'percent': 45.0}, '...","{'leave': {'total': 19202, 'percent': 39.4}, '...","{'leave': {'total': 26511, 'percent': 44.7}, '..."


In [6]:
# Transpose columns and row
df = df.T

In [7]:
df.head()

,0
E07000026,"{'leave': {'total': 31809, 'percent': 58.6}, '..."
E07000032,"{'leave': {'total': 44501, 'percent': 60.3}, '..."
E07000223,"{'leave': {'total': 20315, 'percent': 54.6}, '..."
E07000224,"{'leave': {'total': 56936, 'percent': 62.5}, '..."
S12000033,"{'leave': {'total': 40729, 'percent': 38.9}, '..."


In [8]:
# Reset index so we can rename the regions column later
df.reset_index(inplace=True)

In [10]:
df.head()

,region,0
0,E07000026,"{'leave': {'total': 31809, 'percent': 58.6}, '..."
1,E07000032,"{'leave': {'total': 44501, 'percent': 60.3}, '..."
2,E07000223,"{'leave': {'total': 20315, 'percent': 54.6}, '..."
3,E07000224,"{'leave': {'total': 56936, 'percent': 62.5}, '..."
4,S12000033,"{'leave': {'total': 40729, 'percent': 38.9}, '..."


In [13]:
# Rename index to region
df.rename(columns={'index':'region'}, inplace=True)

In [14]:
df.head()

,region,0
0,E07000026,"{'leave': {'total': 31809, 'percent': 58.6}, '..."
1,E07000032,"{'leave': {'total': 44501, 'percent': 60.3}, '..."
2,E07000223,"{'leave': {'total': 20315, 'percent': 54.6}, '..."
3,E07000224,"{'leave': {'total': 56936, 'percent': 62.5}, '..."
4,S12000033,"{'leave': {'total': 40729, 'percent': 38.9}, '..."


In [15]:
# Rename 0 labeled column to data so we can reference it later
df.rename(columns={0:'data'}, inplace=True)

In [16]:
df.head()

,region,data
0,E07000026,"{'leave': {'total': 31809, 'percent': 58.6}, '..."
1,E07000032,"{'leave': {'total': 44501, 'percent': 60.3}, '..."
2,E07000223,"{'leave': {'total': 20315, 'percent': 54.6}, '..."
3,E07000224,"{'leave': {'total': 56936, 'percent': 62.5}, '..."
4,S12000033,"{'leave': {'total': 40729, 'percent': 38.9}, '..."


In [17]:
# df['region'] is a single column which is a Series.
# Need to convert it into a DataFrame so we can join together 2 DataFrames
# because we can't join a Series with a DataFrame directly
df_region = pd.DataFrame(df['region'])

In [18]:
df_region.head()

,region
0,E07000026
1,E07000032
2,E07000223
3,E07000224
4,S12000033


In [19]:
# Inspect the data column
df['data']

0    {'leave': {'total': 31809, 'percent': 58.6}, '...
1    {'leave': {'total': 44501, 'percent': 60.3}, '...
2    {'leave': {'total': 20315, 'percent': 54.6}, '...
3    {'leave': {'total': 56936, 'percent': 62.5}, '...
4    {'leave': {'total': 40729, 'percent': 38.9}, '...
5    {'leave': {'total': 62516, 'percent': 45.0}, '...
6    {'leave': {'total': 19202, 'percent': 39.4}, '...
7    {'leave': {'total': 26511, 'percent': 44.7}, '...
Name: data, dtype: object

In [20]:
# Flatten the JSON in the data column into a DataFrame
df_data = json_normalize(df['data'])

In [23]:
# Confirm nested JSON converted into their own columns
df_data.head()

,leave.percent,leave.total,name,outcome,remain.percent,remain.total,turnout.percent
0,58.6,31809,Allerdale,L,41.4,22429,72.9
1,60.3,44501,Amber Valley,L,39.7,29319,76.3
2,54.6,20315,Adur,L,45.4,16914,76.4
3,62.5,56936,Arun,L,37.5,34193,77.8
4,38.9,40729,Aberdeen City,R,61.1,63985,67.9


In [74]:
# Join the 2 new DataFrames, df_region and df_data
df = df_region.join(df_data)

In [75]:
df.head()

,region,leave.percent,leave.total,name,outcome,remain.percent,remain.total,turnout.percent
0,E07000026,58.6,31809,Allerdale,L,41.4,22429,72.9
1,E07000032,60.3,44501,Amber Valley,L,39.7,29319,76.3
2,E07000223,54.6,20315,Adur,L,45.4,16914,76.4
3,E07000224,62.5,56936,Arun,L,37.5,34193,77.8
4,S12000033,38.9,40729,Aberdeen City,R,61.1,63985,67.9


In [76]:
# write DataFrame to a csv file
df.to_csv('brexit_json_flattened.csv', index=False)